In [37]:
from pycaret.datasets import get_data
from pycaret.regression import *
import pandas as pd
import csv

In [38]:
df = get_data('data/initial_features_mod')
df.columns

,subject,experimental_condition,rpe,wrist_acc_time,wrist_acc_length,wrist_acc_mean,wrist_acc_rms,wrist_acc_mad,wrist_acc_std,wrist_acc_min,...,upperarm_jerk_rms,upperarm_jerk_mad,upperarm_jerk_std,upperarm_jerk_min,upperarm_jerk_max,upperarm_jerk_med,upperarm_jerk_perc25,upperarm_jerk_perc75,weight,pace
0,1,Condition 2.5-15,0,1.631984e+09,2952,1.017365,1.042456,0.153341,0.227376,0.426780,...,4.277703,2.170631,2.824283,0.014541,19.016887,2.537380,0.980603,4.603277,2.5,15
1,1,Condition 2.5-15,0,1.631984e+09,2414,1.019116,1.046801,0.166215,0.239201,0.481607,...,4.663319,2.295026,3.278950,0.036583,31.471509,2.540472,1.007865,4.469952,2.5,15
2,1,Condition 2.5-15,0,1.631984e+09,2374,1.020968,1.053935,0.183874,0.261596,0.478265,...,4.684816,2.225496,3.102505,0.027138,29.295564,2.826385,1.357187,4.803642,2.5,15
3,1,Condition 2.5-15,0,1.631984e+09,2455,1.015247,1.039529,0.167656,0.223415,0.452296,...,3.675835,1.761217,2.281765,0.024198,18.058972,2.464632,1.135283,4.091388,2.5,15
4,1,Condition 2.5-15,0,1.631984e+09,2502,1.022933,1.051561,0.167432,0.243747,0.396358,...,4.896867,2.419522,3.281058,0.029791,27.380172,2.791626,1.318934,4.934743,2.5,15


Index(['subject', 'experimental_condition', 'rpe', 'wrist_acc_time',
       'wrist_acc_length', 'wrist_acc_mean', 'wrist_acc_rms', 'wrist_acc_mad',
       'wrist_acc_std', 'wrist_acc_min', 'wrist_acc_max', 'wrist_acc_med',
       'wrist_acc_perc25', 'wrist_acc_perc75', 'wrist_jerk_mean',
       'wrist_jerk_rms', 'wrist_jerk_mad', 'wrist_jerk_std', 'wrist_jerk_min',
       'wrist_jerk_max', 'wrist_jerk_med', 'wrist_jerk_perc25',
       'wrist_jerk_perc75', 'trunk_acc_mean', 'trunk_acc_rms', 'trunk_acc_mad',
       'trunk_acc_std', 'trunk_acc_min', 'trunk_acc_max', 'trunk_acc_med',
       'trunk_acc_perc25', 'trunk_acc_perc75', 'trunk_jerk_mean',
       'trunk_jerk_rms', 'trunk_jerk_mad', 'trunk_jerk_std', 'trunk_jerk_min',
       'trunk_jerk_max', 'trunk_jerk_med', 'trunk_jerk_perc25',
       'trunk_jerk_perc75', 'upperarm_acc_mean', 'upperarm_acc_rms',
       'upperarm_acc_mad', 'upperarm_acc_std', 'upperarm_acc_min',
       'upperarm_acc_max', 'upperarm_acc_med', 'upperarm_acc_perc25'

In [42]:
# EXAMPLE RUN
sub = (1, 2, 3)

train = df[~df['subject'].isin(sub)]
test = df[df['subject'].isin(sub)]

reg = setup(data=train, target='rpe', ignore_features=['experimental_condition', 'subject'])
best = compare_models(sort='MAE', n_select=2, include=['rf', 'dt'])

,Description,Value
0,Session id,1269
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(2109, 61)"
4,Transformed data shape,"(2109, 59)"
5,Transformed train set shape,"(1476, 59)"
6,Transformed test set shape,"(633, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.8423,1.6066,1.2583,0.7223,0.4227,0.2677,0.9920
dt,Decision Tree Regressor,0.9069,3.1157,1.7521,0.4602,0.5374,0.3390,0.0370


In [43]:
for model in best:
    print(model)

RandomForestRegressor(n_jobs=-1, random_state=1269)
DecisionTreeRegressor(random_state=1269)


In [45]:
with open('outputs/individual/feature_importance_individual.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Feature", "Value", "Model"])

for model in best:
    # Do this for all columns and get feature importance, save in table, col for: test subjects, model name
    try:  # KNN might need other way of doing...
        importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.feature_importances_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
    except:
        try:
            importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.coef_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
        except:
            importance = pd.DataFrame({'Feature': ['error'], 'Value': [0]})
    importance['Model'] = str(model)
    importance.to_csv('outputs/individual/feature_importance_individual.csv', mode='a', header=False, index=False)

,Feature,Value,Model
0,wrist_acc_time,0.374231,"RandomForestRegressor(n_jobs=-1, random_state=..."
1,upperarm_acc_mean,0.060190,"RandomForestRegressor(n_jobs=-1, random_state=..."
2,upperarm_acc_med,0.037898,"RandomForestRegressor(n_jobs=-1, random_state=..."
3,trunk_acc_mean,0.029806,"RandomForestRegressor(n_jobs=-1, random_state=..."
4,trunk_acc_rms,0.028935,"RandomForestRegressor(n_jobs=-1, random_state=..."
5,upperarm_acc_perc25,0.022836,"RandomForestRegressor(n_jobs=-1, random_state=..."
6,wrist_acc_med,0.019616,"RandomForestRegressor(n_jobs=-1, random_state=..."
7,upperarm_acc_min,0.018986,"RandomForestRegressor(n_jobs=-1, random_state=..."
8,wrist_acc_mean,0.016814,"RandomForestRegressor(n_jobs=-1, random_state=..."
9,upperarm_acc_rms,0.016423,"RandomForestRegressor(n_jobs=-1, random_state=..."
